In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install transformers datasets torch trl peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 14.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 198.5 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu

In [2]:
from kaggle_secrets import UserSecretsClient
import wandb

wandb.login(key=UserSecretsClient().get_secret("wandb"))

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: omarabdelnasser024 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [9]:
dataset["train"]

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 18949
})

In [15]:
from datasets import load_datasetformatting_prompts_func
from transformers import AutoTokenizer

model_name = "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_dataset = dataset["train"]PeftModelForCausalLM

# count token lengths
def count_token_lengths(example):
    return {
        "text_length": len(tokenizer.encode(example["text"], truncation=False)),
        "summary_length": len(tokenizer.encode(example["summary"], truncation=False)),
        "title_length": len(tokenizer.encode(example["title"], truncation=False)),
    }

# Apply to dataset
train_dataset_with_lengths = train_dataset.map(count_token_lengths)

# Get max lengths
max_text_length = max(train_dataset_with_lengths["text_length"])
max_summary_length = max(train_dataset_with_lengths["summary_length"])
max_title_length = max(train_dataset_with_lengths["title_length"])

print("Max token length in 'text':", max_text_length)
print("Max token length in 'summary':", max_summary_length)
print("Max token length in 'title':", max_title_length)


Map:   0%|          | 0/18949 [00:00<?, ? examples/s]

Max token length in 'text': 7962
Max token length in 'summary': 1147
Max token length in 'title': 156


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)


tokenizer_config.json:   0%|          | 0.00/9.68k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

2025-05-10 16:40:31.565031: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746895232.006333      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746895232.135482      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [4]:
legal_summarization_prompt = """Below is a legal document that needs to be summarized. Create a concise summary with an appropriate title that captures the key points and implications of the document.

### Legal Document:
{}

### Title:
{}

### Summary:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    legal_documents = examples["text"]  
    titles = examples["title"] 
    summaries = examples["summary"]  
    
    texts = []
    for document, title, summary in zip(legal_documents, titles, summaries):
        text = legal_summarization_prompt.format(document, title, summary) + EOS_TOKEN
        texts.append(text)
    
    return {"text": texts}


from datasets import load_dataset
dataset = load_dataset("billsum")
dataset = dataset.map(formatting_prompts_func, batched=True)

README.md:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/91.8M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

ca_test-00000-of-00001.parquet:   0%|          | 0.00/6.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Map:   0%|          | 0/18949 [00:00<?, ? examples/s]

Map:   0%|          | 0/3269 [00:00<?, ? examples/s]

Map:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [5]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
)
from trl import SFTTrainer
import torch
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=128,                     
    lora_alpha=256,           
    lora_dropout=0.1,         
    bias="all",             
    task_type="CAUSAL_LM",    
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],)
model = get_peft_model(model, peft_config)

training_args = TrainingArguments(
    output_dir="./results",        
    max_steps = 1000,
    per_device_train_batch_size=2, 
    gradient_accumulation_steps=4,  
    gradient_checkpointing=False,   
    optim="adamw_torch",            
    learning_rate=1e-4,            
    warmup_ratio=0.1,               
    fp16=True,
    logging_strategy="steps",
    logging_steps = 1,
    lr_scheduler_type="linear",
    remove_unused_columns=True,         
    report_to="wandb",                   
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"]
)

trainer.train()

Converting train dataset to ChatML:   0%|          | 0/18949 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/18949 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/18949 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/18949 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
1,1.893800
2,1.912900
3,2.059100
4,2.002900
5,1.926300
6,1.925200
7,1.917500
8,1.896400
9,1.853600
10,1.832700


TrainOutput(global_step=1000, training_loss=1.291771509885788, metrics={'train_runtime': 3894.9707, 'train_samples_per_second': 2.054, 'train_steps_per_second': 0.257, 'total_flos': 2.5618406178816e+16, 'train_loss': 1.291771509885788})

In [6]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
login(token=user_secrets.get_secret("write_token"))


In [7]:
model.save_pretrained("Qwen3-0.6B/California bill legel summarization/lora")
tokenizer.save_pretrained("Qwen3-0.6B/California bill legel summarization/lora")

('Qwen3-0.6B/California bill legel summarization/lora/tokenizer_config.json',
 'Qwen3-0.6B/California bill legel summarization/lora/special_tokens_map.json',
 'Qwen3-0.6B/California bill legel summarization/lora/vocab.json',
 'Qwen3-0.6B/California bill legel summarization/lora/merges.txt',
 'Qwen3-0.6B/California bill legel summarization/lora/added_tokens.json',
 'Qwen3-0.6B/California bill legel summarization/lora/tokenizer.json')

In [8]:
from huggingface_hub import upload_folder, create_repo

folder_path = "/kaggle/working/Qwen3-0.6B/California bill legel summarization/lora"

org_name = "NLUandHelwan" 
model_repo_name = "qwen3-0.6b-california-bill-lora"

# Full repo ID 
repo_id = f"{org_name}/{model_repo_name}"


create_repo(repo_id, repo_type="model", exist_ok=True)

# Upload folder
upload_folder(
    repo_id=repo_id,
    folder_path=folder_path,
    path_in_repo=".",  
    repo_type="model"
)


adapter_model.safetensors:   0%|          | 0.00/323M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/NLUandHelwan/qwen3-0.6b-california-bill-lora/commit/646ae1004311d5d3096a520de3e60856bc2ebf93', commit_message='Upload folder using huggingface_hub', commit_description='', oid='646ae1004311d5d3096a520de3e60856bc2ebf93', pr_url=None, repo_url=RepoUrl('https://huggingface.co/NLUandHelwan/qwen3-0.6b-california-bill-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='NLUandHelwan/qwen3-0.6b-california-bill-lora'), pr_revision=None, pr_num=None)

In [9]:
from peft import PeftModel
base_model_id = "Qwen/Qwen3-0.6B"  
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
base_model = AutoModelForCausalLM.from_pretrained(base_model_id, torch_dtype=torch.float16).to("cuda")
peft_model_id = "NLUandHelwan/qwen3-0.6b-california-bill-lora" 
model = PeftModel.from_pretrained(base_model, peft_model_id).to("cuda")


adapter_config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/323M [00:00<?, ?B/s]